In [1]:
import os
import tarfile

In [2]:
'''
文件数据结构：
data-----
        |
        -----类别1
        |       |
        |       -----患者1
        |       |       |
        |       |       -----文件1.dcm
        |       |       |
        |       |       -----文件2.dcm
        |       |       。。。
        |       |
        |       -----患者2
        |       |       |
        |       |       -----文件1.dcm
        |       |       |
        |       |       -----文件2.dcm
        |       |       。。。
        |       。。。
        |
        |
        -----类别2
        |       |
        |       -----患者1
        |       |       |
        |       |       -----文件1.dcm
        |       |       |
        |       |       -----文件2.dcm
        |       |       。。。
        |       |
        |       -----患者2
        |       |       |
        |       |       -----文件1.dcm
        |       |       |
        |       |       -----文件2.dcm
        |       |       。。。
        |       。。。
        |
        |
        -----类别3
                |
                -----患者1
                |       |
                |       -----文件1.dcm
                |       |
                |       -----文件2.dcm
                |       。。。
                |
                -----患者2
                |       |
                |       -----文件1.dcm
                |       |
                |       -----文件2.dcm
                |       。。。
                。。。
'''


'''
dcm文件解压函数。

param：
- root_folder: 文件夹根路径

return:
- none

'''
# def extract_tar_files(root_folder):
#     # 遍历父文件夹下的所有子文件夹
#     for folder_name in os.listdir(root_folder):
#         folder_path = os.path.join(root_folder, folder_name)
#         # 检查子文件夹是否为目录
#         if os.path.isdir(folder_path):
#             # 遍历当前子文件夹下的所有子子文件夹
#             for subfolder_name in os.listdir(folder_path):
#                 subfolder_path = os.path.join(folder_path, subfolder_name)
#                 # 检查子子文件夹是否为目录
#                 if os.path.isdir(subfolder_path):
#                     # 遍历当前子子文件夹下的所有文件
#                     for file in os.listdir(subfolder_path):
#                         if file.endswith('.tar'):
#                             tar_path = os.path.join(subfolder_path, file)
#                             # 打开 tar 文件
#                             with tarfile.open(tar_path, 'r') as tar:
#                                 # 逐个提取文件并指定提取的目标路径
#                                 for member in tar.getmembers():
#                                     tar.extract(member, path=subfolder_path)
#                             # 解压完后删除 .tar 文件
#                             os.remove(tar_path)

# # 指定根文件夹路径
# root_folder = "D:/CTData/test"
# # 调用函数解压 tar 文件
# extract_tar_files(root_folder)


In [1]:
import os
import cv2
import pydicom
from shutil import copyfile
import numpy as np

In [4]:
'''
归一化函数

param：
- img: 图像路径

return:
- img: 图像路径
'''
def to255(img):
    min_val = img.min()
    max_val = img.max()
    
    # 图像归一化
    img = (img - min_val) / (max_val - min_val + 1e-5)
    img = img * 255
    return img

In [16]:
'''
窗宽、窗位调整函数

param：
- ds: 原始dcm图像路径

return:
- ct_image: 转化后的图像
'''
def normalize_image(ds):
    info20 = ds.RescaleIntercept
    info21 = int(ds.RescaleSlope)
    info22 = ds.pixel_array
    CT = info21 * info22 + info20
    info18 = ds.WindowCenter
    info19 = ds.WindowWidth
    if(isinstance(info18, pydicom.valuerep.DSfloat)):
        info18 = ds.WindowCenter
        info19 = ds.WindowWidth
    elif(isinstance(info18, pydicom.multival.MultiValue)):
        info18 = ds.WindowCenter[0]
        info19 = ds.WindowWidth[0]
    else:
        pass
    CT_min = info18 - info19/2
    CT_max = info18 + info19/2
    CT = np.clip(CT,CT_min,CT_max)

    # 归一化
    ct_image = to255(CT)
    return ct_image
    

In [6]:
'''
dcm转换jpg函数

param：
- input_folder: 原始dcm图像路径
- output_folder: 输出jpg图像路径

return:
- none
'''
def dicom_to_jpg(input_folder, output_folder):
    # 遍历输入文件夹下的所有子文件夹和文件
    for root, dirs, files in os.walk(input_folder):
        # 获取相对于输入文件夹的路径
        relative_path = os.path.relpath(root, input_folder)
        # 构建输出文件夹路径
        output_subfolder = os.path.join(output_folder, relative_path)
        os.makedirs(output_subfolder, exist_ok=True)
        print('正在遍历'+output_subfolder)
        # 遍历当前文件夹下的所有文件
        for file in files:
            if file.endswith('.dcm'):
                try:
                    dicom_path = os.path.join(root, file)
                    output_path = os.path.join(output_subfolder, os.path.splitext(file)[0] + '.png')

                    # 读取 DICOM 文件
                    dicom_data = pydicom.dcmread(dicom_path)
                    # 归一化图像数组
                    normalized_image = normalize_image(dicom_data)
                    # 使用 OpenCV 将归一化后的图像数组保存为 JPG 文件
                    cv2.imwrite(output_path, normalized_image)
                except Exception as e:
                    print(f"处理文件 {dicom_path} 时出错：{e}")
                    pass
        print('遍历完成'+output_subfolder)


In [7]:
'''
文件结构复制函数

param：
- input_folder: 原始dcm图像路径
- output_folder: 输出jpg图像路径

return:
- none

'''
def create_folder_structure(input_folder, output_folder):
    # 遍历输入文件夹下的所有子文件夹
    for root, dirs, files in os.walk(input_folder):
        # 获取相对于输入文件夹的路径
        relative_path = os.path.relpath(root, input_folder)
        # 构建输出文件夹路径
        output_subfolder = os.path.join(output_folder, relative_path)
        os.makedirs(output_subfolder, exist_ok=True)

        # 遍历当前文件夹下的所有文件
        for file in files:
            if not file.endswith('.dcm'):
                input_path = os.path.join(root, file)
                output_path = os.path.join(output_subfolder, file)
                # 复制非 DICOM 文件到输出文件夹
                copyfile(input_path, output_path)


In [17]:
# 输入文件夹路径（包含DICOM文件）
input_folder = 'D:/CTData/test_3'
# 输出文件夹路径（保存转换后的JPEG文件）
output_folder = 'D:/CTData/test_4'


# 将DICOM文件转换为JPEG格式并保存至输出文件夹
dicom_to_jpg(input_folder, output_folder)

# 根据原始文件结构创建相同的文件结构并复制非DICOM文件到输出文件夹
create_folder_structure(input_folder, output_folder)

print("转换完成并保存至 " + output_folder)



正在遍历D:/CTData/test_4\.
遍历完成D:/CTData/test_4\.
正在遍历D:/CTData/test_4\meCT
遍历完成D:/CTData/test_4\meCT
正在遍历D:/CTData/test_4\meCT\0001858059
处理文件 D:/CTData/test_3\meCT\0001858059\70154739.dcm 时出错：'FileDataset' object has no attribute 'RescaleIntercept'
处理文件 D:/CTData/test_3\meCT\0001858059\70154740.dcm 时出错：'FileDataset' object has no attribute 'RescaleIntercept'
处理文件 D:/CTData/test_3\meCT\0001858059\70154741.dcm 时出错：'FileDataset' object has no attribute 'RescaleIntercept'
处理文件 D:/CTData/test_3\meCT\0001858059\70154742.dcm 时出错：'FileDataset' object has no attribute 'RescaleIntercept'
处理文件 D:/CTData/test_3\meCT\0001858059\70154743.dcm 时出错：'FileDataset' object has no attribute 'RescaleIntercept'
处理文件 D:/CTData/test_3\meCT\0001858059\70154744.dcm 时出错：'FileDataset' object has no attribute 'RescaleIntercept'
处理文件 D:/CTData/test_3\meCT\0001858059\70154755.dcm 时出错：'FileDataset' object has no attribute 'RescaleIntercept'
处理文件 D:/CTData/test_3\meCT\0001858059\70154757.dcm 时出错：'FileDataset' object has n